In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_weather(lon, lat):
    print(f"Fetching weather for coordinates: {lon}, {lat}")
    
function = {
    "name": "get_weather",
    "description": "function to get weather information based on the longtitude and latitude of a city",
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "The longitude coordinate of the city"
            },
            "lat": {
                "type": "string",
                "description": "The latitude coordinate of the city"
            }
        },
    },
    "required": ["lon", "lat"]
}

llm = ChatOpenAI(
    temperature=0.1
    ).bind(
        function_call="auto",
        functions=[
            function
        ]
    )
prompt = PromptTemplate.from_template("How's the weather in {city}?")

chain = prompt | llm

response = chain.invoke({
    "city": "San Francisco"
    })

response = response.additional_kwargs["function_call"]["arguments"]

In [5]:
import json

r = json.loads(response)

get_weather(r['lon'], r['lat'])

Fetching weather for coordinates: -122.4194, 37.7749
